In [ ]:
f = dei_loss
f_eval=base_model
x = starting_im
mei_act = mei_activation
random_dir='random'
seed=19
target_level = 0.9
dev_thre=0.05
step_size=4
optim_kwargs={'momentum': 0.9}
gradient_f=gradient_f
transform=None
regularization=None
optim_name='SGD'
optim_kwargs={}
i=0

# Basic checks
if x.dtype != torch.float32:
    raise ValueError('x must be of torch.float32 dtype')
x = x.detach().clone()  # to avoid changing original
x.requires_grad_()

# Declare optimizer
if optim_name == 'SGD':
    optimizer = optim.SGD([x], lr=step_size, **optim_kwargs)
elif optim_name == 'Adam':
    optimizer = optim.Adam([x], lr=step_size, **optim_kwargs)
else:
    raise ValueError("Expected optim_name to be 'SGD' or 'Adam'")

# Run gradient ascent
fevals = []  # to store function evaluations
reg_terms = []  # to store regularization function evaluations
saved_xs = []  # to store xs (ignored if save_iters is None)

torch.manual_seed(4)

# Zero gradients
if x.grad is not None:
    x.grad.zero_()

# Transform input
transformed_x = x if transform is None else transform(x, iteration=i)

# f(x)
feval = f(transformed_x)
fevals.append(feval.item())

# Regularization
if regularization is not None:
    reg_term = regularization(transformed_x, iteration=i)
    reg_terms.append(reg_term.item())
else:
    reg_term = 0
    
# Random walk or optimize back to the target activation level
dirs = []
rand_grad = []
if random_dir is None:
    (-feval + reg_term).backward()
    if gradient_f is not None:
        x.grad = gradient_f(x.grad, iteration=i)
    x.grad = x.grad / torch.norm(x.grad)

elif (f_eval(transformed_x) >= mei_act * (target_level - dev_thre)) and (f_eval(transformed_x) <= mei_act * (target_level + dev_thre)):
    direction = torch.randn(x.shape).cuda()
    if random_dir == 'random':
        x.grad = direction
        dirs.append(gradient_f(x.grad, iteration=i) / torch.norm(gradient_f(x.grad, iteration=i)))

        if gradient_f is not None:
            x.grad = gradient_f(x.grad, iteration=i)
        x.grad = x.grad / torch.norm(x.grad)
        rand_grad.append(x.grad)